In [ ]:
#%run NB01-Load.ipynb

In [1]:
import collections
import gc
import IPython
import lightgbm as lgb
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.plotly as py
import random 
#import seaborn as sb
import seaborn as sns
#import seaborn as snss
import scipy
import scipy as sp
import sklearn
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sys
import warnings

from datetime import date
from IPython.display import HTML
from lightgbm import LGBMClassifier
from lightgbm import plot_importance
from pandas import DataFrame
from pandas import Series
from patsy import dmatrices
from plotly import tools
from plotly.offline import init_notebook_mode
from plotly.offline import iplot
from pylab import rcParams
from sklearn import datasets
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
#from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from xgboost import XGBClassifier
from wordcloud import WordCloud

init_notebook_mode(connected=True)
plt.rcParams.update({'figure.max_open_warning': 200})
# Suppress warnings
warnings.filterwarnings("ignore")

# In a notebook environment, display the plots inline
%matplotlib inline
# Set some parameters to apply to all plots. These can be overridden in each plot if desired
# Plot size to 14" x 7"
matplotlib.rc('figure', figsize = (14, 7))
# Font size to 14
matplotlib.rc('font', size = 14)
# Do not display top and right frame lines
matplotlib.rc('axes.spines', top = False, right = False)
# Remove grid lines
matplotlib.rc('axes', grid = False)
# Set backgound color to white
matplotlib.rc('axes', facecolor = 'white')

np.set_printoptions(threshold=np.nan)
np.set_printoptions(suppress=True, formatter={'float_kind':'{:f}'.format})

#import C:/Development/kaggle--home-credit-default-risk/rand_jitter
#import C:/Development/kaggle--home-credit-default-risk/draw_feature_distribution
import sys
# sys.path.insert(0, 'C:/Development/kaggle--home-credit-default-risk/') # ~= sys.path.prepend
sys.path.append('C:/Development/kaggle--home-credit-default-risk/')
# import rand_jitter
# import draw_feature_distribution
##from rand_jitter import * # NOTE: added directly to draw_feature_distribution_v2
# from draw_feature_distribution import *
# from draw_feature_distribution_v1 import *
from draw_feature_distribution_v2 import *

# C:\Users\jbalcomb\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning:
# The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.


In [2]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("scikit-learn version: {}".format(sklearn.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("IPython version: {}".format(IPython.__version__))

Python version: 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
pandas version: 0.20.3
NumPy version: 1.14.3
SciPy version: 1.1.0
scikit-learn version: 0.19.1
matplotlib version: 2.1.0
IPython version: 6.1.0


In [3]:
data_file_path = 'C:/Development/kaggle--home-credit-default-risk/data/'

In [4]:
application_train = pd.read_table(data_file_path + 'application_train.csv', sep=',')

In [ ]:
#application_train = application_train.fillna(np.nan)

In [ ]:
#df = application_train

In [5]:
application_train.shape

(307511, 122)

In [10]:
application_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [11]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
application_train.tail()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
307510,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0


In [13]:
application_train.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [14]:
application_train_column_names = application_train.columns.tolist()
application_train_column_names

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_A

In [16]:
application_train__dtypes = application_train.dtypes

In [17]:
application_train__dtypes.to_csv('application_train__dtypes.csv')

In [18]:
application_train__dtypes

SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
CNT_CHILDREN                    int64
AMT_INCOME_TOTAL              float64
AMT_CREDIT                    float64
AMT_ANNUITY                   float64
AMT_GOODS_PRICE               float64
NAME_TYPE_SUITE                object
NAME_INCOME_TYPE               object
NAME_EDUCATION_TYPE            object
NAME_FAMILY_STATUS             object
NAME_HOUSING_TYPE              object
REGION_POPULATION_RELATIVE    float64
DAYS_BIRTH                      int64
DAYS_EMPLOYED                   int64
DAYS_REGISTRATION             float64
DAYS_ID_PUBLISH                 int64
OWN_CAR_AGE                   float64
FLAG_MOBIL                      int64
FLAG_EMP_PHONE                  int64
FLAG_WORK_PHONE                 int64
FLAG_CONT_MOBILE                int64
FLAG_PHONE  

In [19]:
application_train.columns.to_series().groupby(application_train.dtypes).groups

{dtype('int64'): Index(['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
        'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE',
        'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'REGION_RATING_CLIENT',
        'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',
        'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
        'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
        'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2',
        'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
        'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
        'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
        'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
        'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
        'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
        'FLAG_DOCUMENT_21'],
       dtype='object'),
 dtype('float6

In [21]:
# application_train.select_dtypes(include=['bool'])
list(application_train.select_dtypes(include=['bool']).columns)

[]

In [22]:
list(application_train.select_dtypes(include=['int64']).columns)

['SK_ID_CURR',
 'TARGET',
 'CNT_CHILDREN',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_ID_PUBLISH',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']

In [23]:
list(application_train.select_dtypes(include=['float64']).columns)

['AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_REGISTRATION',
 'OWN_CAR_AGE',
 'CNT_FAM_MEMBERS',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_M

In [24]:
list(application_train.select_dtypes(include=['O']).columns)

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [40]:
# List of unique values in the df['name'] column
# df.name.unique()
application_train.TARGET.unique()

array([1, 0], dtype=int64)

In [41]:
application_train['TARGET'].unique()

array([1, 0], dtype=int64)

In [42]:
# Number of unique values in the df['name'] column
application_train.TARGET.nunique()

2

In [43]:
application_train['TARGET'].nunique()

2

In [44]:
application_train['TARGET'].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

https://stackoverflow.com/questions/45759966/counting-unique-values-in-a-column-in-pandas-dataframe-like-in-qlik

Count distict values, use nunique:
df['hID'].nunique()
Count only non-null values, use count:
df['hID'].count()
Count total values including null values, use size attribute:
df['hID'].size
To add condition...
Use boolean indexing:
df.loc[df['mID']=='A','hID'].agg(['nunique','count','size'])
Or, using query:
df.query('mID == "A"')['hID'].agg(['nunique','count','size'])

New in pandas 0.20.0 pd.DataFrame.agg
df.agg(['count', 'size', 'nunique'])
You've always been able to do an agg within a groupby. I used stack at the end because I like the presentation better.
df.groupby('mID').agg(['count', 'size', 'nunique']).stack()


https://stackoverflow.com/questions/45125408/how-to-count-the-distinct-values-across-a-column-in-pandas

df[['Company', 'Date']].drop_duplicates()['Company'].value_counts()
df.groupby('Company')['Date'].nunique()


https://stackoverflow.com/questions/48162201/pandas-number-of-unique-values-and-sort-by-the-number-of-unique

df = df.groupby('A')['B'].nunique().sort_values(ascending=False).reset_index(name='count')
print (df)


https://stackoverflow.com/questions/38309729/count-unique-values-with-pandas-per-groups/38309823

You can retain the column name like this:
df = df.groupby(by='domain', as_index=False).agg({'ID': pd.Series.nunique})
The difference is that 'nunique()' returns a Series and 'agg()' returns a DataFrame.


In [49]:
len(application_train.index)

307511

In [51]:
application_train.shape[0]

307511

In [50]:
len(application_train.columns)

122

In [ ]:
application_train.shape[1]

In [53]:
application_train__row_count, application_train__column_count = application_train.shape

In [54]:
application_train__row_count

307511

In [55]:
application_train__column_count

122

In [67]:
for column_name in application_train_column_names:
    number_of_unique_values = application_train[column_name].nunique()
    print ('Column Name: ', column_name, 'Number of Unique Values: ', number_of_unique_values, 'Cardinality: ', round(number_of_unique_values/application_train__row_count,2))

Column Name:  SK_ID_CURR Number of Unique Values:  307511 Cardinality:  1.0
Column Name:  TARGET Number of Unique Values:  2 Cardinality:  0.0
Column Name:  NAME_CONTRACT_TYPE Number of Unique Values:  2 Cardinality:  0.0
Column Name:  CODE_GENDER Number of Unique Values:  3 Cardinality:  0.0
Column Name:  FLAG_OWN_CAR Number of Unique Values:  2 Cardinality:  0.0
Column Name:  FLAG_OWN_REALTY Number of Unique Values:  2 Cardinality:  0.0
Column Name:  CNT_CHILDREN Number of Unique Values:  15 Cardinality:  0.0
Column Name:  AMT_INCOME_TOTAL Number of Unique Values:  2548 Cardinality:  0.01
Column Name:  AMT_CREDIT Number of Unique Values:  5603 Cardinality:  0.02
Column Name:  AMT_ANNUITY Number of Unique Values:  13672 Cardinality:  0.04
Column Name:  AMT_GOODS_PRICE Number of Unique Values:  1002 Cardinality:  0.0
Column Name:  NAME_TYPE_SUITE Number of Unique Values:  7 Cardinality:  0.0
Column Name:  NAME_INCOME_TYPE Number of Unique Values:  8 Cardinality:  0.0
Column Name:  NAME

Column Name:  FLAG_DOCUMENT_21 Number of Unique Values:  2 Cardinality:  0.0
Column Name:  AMT_REQ_CREDIT_BUREAU_HOUR Number of Unique Values:  5 Cardinality:  0.0
Column Name:  AMT_REQ_CREDIT_BUREAU_DAY Number of Unique Values:  9 Cardinality:  0.0
Column Name:  AMT_REQ_CREDIT_BUREAU_WEEK Number of Unique Values:  9 Cardinality:  0.0
Column Name:  AMT_REQ_CREDIT_BUREAU_MON Number of Unique Values:  24 Cardinality:  0.0
Column Name:  AMT_REQ_CREDIT_BUREAU_QRT Number of Unique Values:  11 Cardinality:  0.0
Column Name:  AMT_REQ_CREDIT_BUREAU_YEAR Number of Unique Values:  25 Cardinality:  0.0


In [69]:
dict((column_name,None) for column_name in application_train_column_names)

{'AMT_ANNUITY': None,
 'AMT_CREDIT': None,
 'AMT_GOODS_PRICE': None,
 'AMT_INCOME_TOTAL': None,
 'AMT_REQ_CREDIT_BUREAU_DAY': None,
 'AMT_REQ_CREDIT_BUREAU_HOUR': None,
 'AMT_REQ_CREDIT_BUREAU_MON': None,
 'AMT_REQ_CREDIT_BUREAU_QRT': None,
 'AMT_REQ_CREDIT_BUREAU_WEEK': None,
 'AMT_REQ_CREDIT_BUREAU_YEAR': None,
 'APARTMENTS_AVG': None,
 'APARTMENTS_MEDI': None,
 'APARTMENTS_MODE': None,
 'BASEMENTAREA_AVG': None,
 'BASEMENTAREA_MEDI': None,
 'BASEMENTAREA_MODE': None,
 'CNT_CHILDREN': None,
 'CNT_FAM_MEMBERS': None,
 'CODE_GENDER': None,
 'COMMONAREA_AVG': None,
 'COMMONAREA_MEDI': None,
 'COMMONAREA_MODE': None,
 'DAYS_BIRTH': None,
 'DAYS_EMPLOYED': None,
 'DAYS_ID_PUBLISH': None,
 'DAYS_LAST_PHONE_CHANGE': None,
 'DAYS_REGISTRATION': None,
 'DEF_30_CNT_SOCIAL_CIRCLE': None,
 'DEF_60_CNT_SOCIAL_CIRCLE': None,
 'ELEVATORS_AVG': None,
 'ELEVATORS_MEDI': None,
 'ELEVATORS_MODE': None,
 'EMERGENCYSTATE_MODE': None,
 'ENTRANCES_AVG': None,
 'ENTRANCES_MEDI': None,
 'ENTRANCES_MODE': Non

In [70]:
dict((column_name,application_train__row_count) for column_name in application_train_column_names)

{'AMT_ANNUITY': 307511,
 'AMT_CREDIT': 307511,
 'AMT_GOODS_PRICE': 307511,
 'AMT_INCOME_TOTAL': 307511,
 'AMT_REQ_CREDIT_BUREAU_DAY': 307511,
 'AMT_REQ_CREDIT_BUREAU_HOUR': 307511,
 'AMT_REQ_CREDIT_BUREAU_MON': 307511,
 'AMT_REQ_CREDIT_BUREAU_QRT': 307511,
 'AMT_REQ_CREDIT_BUREAU_WEEK': 307511,
 'AMT_REQ_CREDIT_BUREAU_YEAR': 307511,
 'APARTMENTS_AVG': 307511,
 'APARTMENTS_MEDI': 307511,
 'APARTMENTS_MODE': 307511,
 'BASEMENTAREA_AVG': 307511,
 'BASEMENTAREA_MEDI': 307511,
 'BASEMENTAREA_MODE': 307511,
 'CNT_CHILDREN': 307511,
 'CNT_FAM_MEMBERS': 307511,
 'CODE_GENDER': 307511,
 'COMMONAREA_AVG': 307511,
 'COMMONAREA_MEDI': 307511,
 'COMMONAREA_MODE': 307511,
 'DAYS_BIRTH': 307511,
 'DAYS_EMPLOYED': 307511,
 'DAYS_ID_PUBLISH': 307511,
 'DAYS_LAST_PHONE_CHANGE': 307511,
 'DAYS_REGISTRATION': 307511,
 'DEF_30_CNT_SOCIAL_CIRCLE': 307511,
 'DEF_60_CNT_SOCIAL_CIRCLE': 307511,
 'ELEVATORS_AVG': 307511,
 'ELEVATORS_MEDI': 307511,
 'ELEVATORS_MODE': 307511,
 'EMERGENCYSTATE_MODE': 307511,
 'ENT

In [72]:
# https://stackoverflow.com/questions/3869487/how-do-i-create-a-dictionary-with-keys-from-a-list-and-values-defaulting-to-say
# Generator expressions avoid the memory overhead of populating the whole list.
# https://stackoverflow.com/questions/2241891/how-to-initialize-a-dict-with-keys-from-a-list-and-empty-value-in-python
# dict.fromkeys(keys_list)
# Be careful with initializing to something mutable: If you call, e.g., dict.fromkeys([1, 2, 3], []), all of the keys are mapped to the same list, and modifying one will modify them all.
# 
# dict-comprehension solution
# keys = [1,2,3,5,6,7]
# {key: None for key in keys}
#> {1: None, 2: None, 3: None, 5: None, 6: None, 7: None}
# 
# Using a dict-comp also allows the value to be the result of calling a function
#   (which could be passed the key as an argument, if desired)

In [78]:
columns_row_count = {column_name:application_train__row_count for column_name in application_train_column_names}

In [79]:
columns_row_count

{'AMT_ANNUITY': 307511,
 'AMT_CREDIT': 307511,
 'AMT_GOODS_PRICE': 307511,
 'AMT_INCOME_TOTAL': 307511,
 'AMT_REQ_CREDIT_BUREAU_DAY': 307511,
 'AMT_REQ_CREDIT_BUREAU_HOUR': 307511,
 'AMT_REQ_CREDIT_BUREAU_MON': 307511,
 'AMT_REQ_CREDIT_BUREAU_QRT': 307511,
 'AMT_REQ_CREDIT_BUREAU_WEEK': 307511,
 'AMT_REQ_CREDIT_BUREAU_YEAR': 307511,
 'APARTMENTS_AVG': 307511,
 'APARTMENTS_MEDI': 307511,
 'APARTMENTS_MODE': 307511,
 'BASEMENTAREA_AVG': 307511,
 'BASEMENTAREA_MEDI': 307511,
 'BASEMENTAREA_MODE': 307511,
 'CNT_CHILDREN': 307511,
 'CNT_FAM_MEMBERS': 307511,
 'CODE_GENDER': 307511,
 'COMMONAREA_AVG': 307511,
 'COMMONAREA_MEDI': 307511,
 'COMMONAREA_MODE': 307511,
 'DAYS_BIRTH': 307511,
 'DAYS_EMPLOYED': 307511,
 'DAYS_ID_PUBLISH': 307511,
 'DAYS_LAST_PHONE_CHANGE': 307511,
 'DAYS_REGISTRATION': 307511,
 'DEF_30_CNT_SOCIAL_CIRCLE': 307511,
 'DEF_60_CNT_SOCIAL_CIRCLE': 307511,
 'ELEVATORS_AVG': 307511,
 'ELEVATORS_MEDI': 307511,
 'ELEVATORS_MODE': 307511,
 'EMERGENCYSTATE_MODE': 307511,
 'ENT

In [80]:
columns_number_of_unique_values = {column_name:None for column_name in application_train_column_names}

In [81]:
columns_number_of_unique_values

{'AMT_ANNUITY': None,
 'AMT_CREDIT': None,
 'AMT_GOODS_PRICE': None,
 'AMT_INCOME_TOTAL': None,
 'AMT_REQ_CREDIT_BUREAU_DAY': None,
 'AMT_REQ_CREDIT_BUREAU_HOUR': None,
 'AMT_REQ_CREDIT_BUREAU_MON': None,
 'AMT_REQ_CREDIT_BUREAU_QRT': None,
 'AMT_REQ_CREDIT_BUREAU_WEEK': None,
 'AMT_REQ_CREDIT_BUREAU_YEAR': None,
 'APARTMENTS_AVG': None,
 'APARTMENTS_MEDI': None,
 'APARTMENTS_MODE': None,
 'BASEMENTAREA_AVG': None,
 'BASEMENTAREA_MEDI': None,
 'BASEMENTAREA_MODE': None,
 'CNT_CHILDREN': None,
 'CNT_FAM_MEMBERS': None,
 'CODE_GENDER': None,
 'COMMONAREA_AVG': None,
 'COMMONAREA_MEDI': None,
 'COMMONAREA_MODE': None,
 'DAYS_BIRTH': None,
 'DAYS_EMPLOYED': None,
 'DAYS_ID_PUBLISH': None,
 'DAYS_LAST_PHONE_CHANGE': None,
 'DAYS_REGISTRATION': None,
 'DEF_30_CNT_SOCIAL_CIRCLE': None,
 'DEF_60_CNT_SOCIAL_CIRCLE': None,
 'ELEVATORS_AVG': None,
 'ELEVATORS_MEDI': None,
 'ELEVATORS_MODE': None,
 'EMERGENCYSTATE_MODE': None,
 'ENTRANCES_AVG': None,
 'ENTRANCES_MEDI': None,
 'ENTRANCES_MODE': Non

In [82]:
for column_name in columns_number_of_unique_values:
    columns_number_of_unique_values[column_name] = application_train[column_name].nunique()

In [83]:
columns_number_of_unique_values

{'AMT_ANNUITY': 13672,
 'AMT_CREDIT': 5603,
 'AMT_GOODS_PRICE': 1002,
 'AMT_INCOME_TOTAL': 2548,
 'AMT_REQ_CREDIT_BUREAU_DAY': 9,
 'AMT_REQ_CREDIT_BUREAU_HOUR': 5,
 'AMT_REQ_CREDIT_BUREAU_MON': 24,
 'AMT_REQ_CREDIT_BUREAU_QRT': 11,
 'AMT_REQ_CREDIT_BUREAU_WEEK': 9,
 'AMT_REQ_CREDIT_BUREAU_YEAR': 25,
 'APARTMENTS_AVG': 2339,
 'APARTMENTS_MEDI': 1148,
 'APARTMENTS_MODE': 760,
 'BASEMENTAREA_AVG': 3780,
 'BASEMENTAREA_MEDI': 3772,
 'BASEMENTAREA_MODE': 3841,
 'CNT_CHILDREN': 15,
 'CNT_FAM_MEMBERS': 17,
 'CODE_GENDER': 3,
 'COMMONAREA_AVG': 3181,
 'COMMONAREA_MEDI': 3202,
 'COMMONAREA_MODE': 3128,
 'DAYS_BIRTH': 17460,
 'DAYS_EMPLOYED': 12574,
 'DAYS_ID_PUBLISH': 6168,
 'DAYS_LAST_PHONE_CHANGE': 3773,
 'DAYS_REGISTRATION': 15688,
 'DEF_30_CNT_SOCIAL_CIRCLE': 10,
 'DEF_60_CNT_SOCIAL_CIRCLE': 9,
 'ELEVATORS_AVG': 257,
 'ELEVATORS_MEDI': 46,
 'ELEVATORS_MODE': 26,
 'EMERGENCYSTATE_MODE': 2,
 'ENTRANCES_AVG': 285,
 'ENTRANCES_MEDI': 46,
 'ENTRANCES_MODE': 30,
 'EXT_SOURCE_1': 114584,
 'EXT_SOU

In [84]:
columns_cardinality = {column_name:None for column_name in application_train_column_names}

In [85]:
columns_cardinality

{'AMT_ANNUITY': None,
 'AMT_CREDIT': None,
 'AMT_GOODS_PRICE': None,
 'AMT_INCOME_TOTAL': None,
 'AMT_REQ_CREDIT_BUREAU_DAY': None,
 'AMT_REQ_CREDIT_BUREAU_HOUR': None,
 'AMT_REQ_CREDIT_BUREAU_MON': None,
 'AMT_REQ_CREDIT_BUREAU_QRT': None,
 'AMT_REQ_CREDIT_BUREAU_WEEK': None,
 'AMT_REQ_CREDIT_BUREAU_YEAR': None,
 'APARTMENTS_AVG': None,
 'APARTMENTS_MEDI': None,
 'APARTMENTS_MODE': None,
 'BASEMENTAREA_AVG': None,
 'BASEMENTAREA_MEDI': None,
 'BASEMENTAREA_MODE': None,
 'CNT_CHILDREN': None,
 'CNT_FAM_MEMBERS': None,
 'CODE_GENDER': None,
 'COMMONAREA_AVG': None,
 'COMMONAREA_MEDI': None,
 'COMMONAREA_MODE': None,
 'DAYS_BIRTH': None,
 'DAYS_EMPLOYED': None,
 'DAYS_ID_PUBLISH': None,
 'DAYS_LAST_PHONE_CHANGE': None,
 'DAYS_REGISTRATION': None,
 'DEF_30_CNT_SOCIAL_CIRCLE': None,
 'DEF_60_CNT_SOCIAL_CIRCLE': None,
 'ELEVATORS_AVG': None,
 'ELEVATORS_MEDI': None,
 'ELEVATORS_MODE': None,
 'EMERGENCYSTATE_MODE': None,
 'ENTRANCES_AVG': None,
 'ENTRANCES_MEDI': None,
 'ENTRANCES_MODE': Non

In [86]:
for column_name in columns_cardinality:
    columns_cardinality[column_name] = columns_number_of_unique_values[column_name]/application_train__row_count

In [87]:
columns_cardinality

{'AMT_ANNUITY': 0.04446019817177272,
 'AMT_CREDIT': 0.01822048642162394,
 'AMT_GOODS_PRICE': 0.0032584200239991414,
 'AMT_INCOME_TOTAL': 0.008285882456237337,
 'AMT_REQ_CREDIT_BUREAU_DAY': 2.9267245724543187e-05,
 'AMT_REQ_CREDIT_BUREAU_HOUR': 1.6259580958079547e-05,
 'AMT_REQ_CREDIT_BUREAU_MON': 7.804598859878183e-05,
 'AMT_REQ_CREDIT_BUREAU_QRT': 3.577107810777501e-05,
 'AMT_REQ_CREDIT_BUREAU_WEEK': 2.9267245724543187e-05,
 'AMT_REQ_CREDIT_BUREAU_YEAR': 8.129790479039775e-05,
 'APARTMENTS_AVG': 0.007606231972189612,
 'APARTMENTS_MEDI': 0.0037331997879750645,
 'APARTMENTS_MODE': 0.0024714563056280913,
 'BASEMENTAREA_AVG': 0.01229224320430814,
 'BASEMENTAREA_MEDI': 0.012266227874775212,
 'BASEMENTAREA_MODE': 0.01249061009199671,
 'CNT_CHILDREN': 4.8778742874238645e-05,
 'CNT_FAM_MEMBERS': 5.528257525747047e-05,
 'CODE_GENDER': 9.755748574847729e-06,
 'COMMONAREA_AVG': 0.010344345405530208,
 'COMMONAREA_MEDI': 0.010412635645554143,
 'COMMONAREA_MODE': 0.010171993847374565,
 'DAYS_BIRTH'

In [96]:
for column_name in columns_number_of_unique_values:
    if columns_number_of_unique_values[column_name] == 2:
        print (column_name, 'dtype: ', application_train[column_name].dtype)

TARGET dtype:  int64
NAME_CONTRACT_TYPE dtype:  object
FLAG_OWN_CAR dtype:  object
FLAG_OWN_REALTY dtype:  object
FLAG_MOBIL dtype:  int64
FLAG_EMP_PHONE dtype:  int64
FLAG_WORK_PHONE dtype:  int64
FLAG_CONT_MOBILE dtype:  int64
FLAG_PHONE dtype:  int64
FLAG_EMAIL dtype:  int64
REG_REGION_NOT_LIVE_REGION dtype:  int64
REG_REGION_NOT_WORK_REGION dtype:  int64
LIVE_REGION_NOT_WORK_REGION dtype:  int64
REG_CITY_NOT_LIVE_CITY dtype:  int64
REG_CITY_NOT_WORK_CITY dtype:  int64
LIVE_CITY_NOT_WORK_CITY dtype:  int64
EMERGENCYSTATE_MODE dtype:  object
FLAG_DOCUMENT_2 dtype:  int64
FLAG_DOCUMENT_3 dtype:  int64
FLAG_DOCUMENT_4 dtype:  int64
FLAG_DOCUMENT_5 dtype:  int64
FLAG_DOCUMENT_6 dtype:  int64
FLAG_DOCUMENT_7 dtype:  int64
FLAG_DOCUMENT_8 dtype:  int64
FLAG_DOCUMENT_9 dtype:  int64
FLAG_DOCUMENT_10 dtype:  int64
FLAG_DOCUMENT_11 dtype:  int64
FLAG_DOCUMENT_12 dtype:  int64
FLAG_DOCUMENT_13 dtype:  int64
FLAG_DOCUMENT_14 dtype:  int64
FLAG_DOCUMENT_15 dtype:  int64
FLAG_DOCUMENT_16 dtype: 

In [ ]:
"""
# http://pbpython.com/pandas-list-dict.html
The “default” manner to create a DataFrame from python is to use a list of dictionaries.
In this case each dictionary key is used for the column headings.
A default index will be created automatically:
sales = [{'account': 'Jones LLC', 'Jan': 150, 'Feb': 200, 'Mar': 140},
         {'account': 'Alpha Co',  'Jan': 200, 'Feb': 210, 'Mar': 215},
         {'account': 'Blue Inc',  'Jan': 50,  'Feb': 90,  'Mar': 95 }]
df = pd.DataFrame(sales)

"""

In [56]:
empty_dictionary = {}
empty_dictionary

{}

In [58]:
any(empty_dictionary)

False

In [59]:
len(empty_dictionary)

0

In [64]:
bool(empty_dictionary)

False

In [61]:
falsy_dictionary = {0:False}
falsy_dictionary

{0: False}

In [62]:
any(falsy_dictionary)

False

In [63]:
len(falsy_dictionary)

1

In [65]:
bool(falsy_dictionary)

True

In [ ]:
"""
# https://stackoverflow.com/questions/23177439/python-checking-if-a-dictionary-is-empty-doesnt-seem-to-work
test_dict = {}

# Option 1
if not test_dict:
    print "Dict is Empty"

# Option 2
if not bool(test_dict):
    print "Dict is Empty"

# Option 3
if len(test_dict) == 0:
    print "Dict is Empty"

# The first test in the answer above is true not only if the dict exists and is empty, but also if test_dict is None.
# So use this test only when you know that the dict object exists (or when the difference does not matter).
# The second way also has that behavior.
# Only the third way barks if test_dict is None.
"""

In [ ]:
application_train_as_objects = pd.read_table(path + 'application_train.csv', sep=',', dtype=object)

In [ ]:
application_train_column_names

In [ ]:
# https://stackoverflow.com/questions/18667410/how-can-i-check-if-a-string-only-contains-letters-in-python

if string.isalpha():
    print("It's all letters")

# NOTE: using [A-Za-z ]+ will not match names with non ASCII letterss
# NOTE: using \w includes digits
import re
def only_letters(tested_string):
    #match = re.match("^[ABCDEFGHJKLM]*$", tested_string)
    #match = re.match("^[A-HJ-M]*$", tested_string)
    match = re.match("^[A-Za-z]*$", tested_string)
    return match is not None

import re
def only_letters(string):
    return re.match(r'[a-z\s]+$',string,2) # JWB: What's with the ",2"?

def only_letters(string):
    return all(letter.isalpha() for letter in string)

def only_letters(s):
    for c in s:
        cat = unicodedata.category(c)
        # Lu: Category: Letter, Uppercase (https://codepoints.net/search?gc=Lu)
        # Ll: Category: Letter, Lowercase (https://codepoints.net/search?gc=Ll)
        # Lt: Category: Letter, Titlecase (https://codepoints.net/search?gc=Lt)
        # Lm: Category: Letter, Modifier  (https://codepoints.net/search?gc=Lm)
        # Lo: Category: Letter, Other     (https://codepoints.net/search?gc=Lo)
        # Latin-1: There are no Lm or Lt category codepoints in the Latin-1 subset of Unicode and only 2 Lo characters, ª (U+00AA) and º (U+00BA), Feminine and Masculine Ordinal Indicator).
        if cat not in ('Lu','Ll','Lo'):
            return False
    return True

# https://stackoverflow.com/questions/29460405/checking-if-string-is-only-letters-and-spaces-python
# To require that the string contains only alphas and spaces:
if all(x.isalpha() or x.isspace() for x in string):
    print("Only alphabetical letters and spaces: yes")
else:
    print("Only alphabetical letters and spaces: no")
# To require that the string contains at least one alpha and at least one space:
if any(x.isalpha() for x in string) and any(x.isspace() for x in string):
# To require that the string contains at least one alpha, at least one space, and only alphas and spaces:
if (any(x.isalpha() for x in string)
    and any(x.isspace() for x in string)
    and all(x.isalpha() or x.isspace() for x in string)):


In [ ]:
#locale.getlocale()
#> NameError: name 'locale' is not defined

In [ ]:
'dog'.isalpha()

In [ ]:
'äöå'.isalpha()

In [ ]:
'привіт'.isalpha()

In [ ]:
u'привіт'.isalpha()

In [ ]:
repr('äöå')

In [ ]:
ascii('äöå')

In [ ]:
# https://stackoverflow.com/questions/4286637/python-isalpha-and-scandics
# http://en.wikipedia.org/wiki/Windows-1252
s = '\xe4\xf6\xe5'
import unicodedata
for c in s:
    u = c.decode('1252')
    print (ascii(c), ascii(u), unicodedata.name(u, '<no name>'))
#'\xe4' u'\xe4' LATIN SMALL LETTER A WITH DIAERESIS
#'\xf6' u'\xf6' LATIN SMALL LETTER O WITH DIAERESIS
#'\xe5' u'\xe5' LATIN SMALL LETTER A WITH RING ABOVE
s.isalpha()

In [ ]:
# The isalpha() methods returns “True” if all characters in the string are alphabets, Otherwise, It returns “False”.
# This function is used to check if the argument contains any alphabets characters such as:
#     ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
# Both uppercase and lowercase alphabets return “True”
# Space is not considered to be alphabet, therefore it returns “False”

def is_plain_text(string):
    if string != 0:
            # require that the string contains only alphas
            #if all(character.isalpha() for character in string)
            # require that the string contains only alphas and spaces
            if all(character.isalpha() or character.isspace() for character in string):
                return True
            else:
                return False
    else:
            return None

In [ ]:
is_plain_text(application_train_as_objects['SK_ID_CURR'])
#TypeError: 'numpy.bool_' object is not iterable

In [ ]:
application_train_as_objects['SK_ID_CURR'].dtype

In [ ]:
application_train_as_objects['SK_ID_CURR'].str.isdigit().any()

In [ ]:
application_train_as_objects['SK_ID_CURR'].str.isdigit().all()

In [ ]:
application_train_as_objects['SK_ID_CURR'].str.isalpha().any()

In [ ]:
application_train_as_objects['SK_ID_CURR'].str.isalpha().all()

In [ ]:
df[pd.to_numeric(df.A, errors='coerce').notnull()]

In [ ]:
application_train['SK_ID_CURR'].count()

In [ ]:
application_train['SK_ID_CURR'].value_counts()

In [ ]:
# checking for uniqueness:
print(len(application_train['SK_ID_CURR'].unique()))

In [ ]:
# How many classes
application_train['TARGET'].nunique()

In [ ]:
# Distribution of those classes
application_train['TARGET'].value_counts(dropna=False)

In [ ]:
dtypes = application_train.dtypes
dtypes = dtypes[dtypes != 'object']
features = list(set(dtypes.index) - set(['TARGET']))
len(features)

In [ ]:
X = df[features]
y = df['TARGET']

In [ ]:
id_column = ['SK_ID_CURR']

In [ ]:
target_column = ['TARGET']

In [ ]:
boolean_columns = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']

In [ ]:
categorical_columns = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']

In [ ]:
object_columns = []

In [ ]:
numerical_columns = []

In [ ]:
int_columns = []

In [ ]:
float_columns = []

In [ ]:
datetime_columns = []

In [ ]:
date_columns = []

In [ ]:
nondata_columns = ['SK_ID_CURR']

In [ ]:
target_distribution = application_train['TARGET'].value_counts()
target_distribution.plot.pie(figsize=(10, 10),
                             title='Target Distribution',
                             fontsize=15, 
                             legend=True,
                             autopct=lambda v: "{:0.1f}%".format(v))

In [97]:
total_nans = application_train.isnull().sum()
total_nans

SK_ID_CURR                         0
TARGET                             0
NAME_CONTRACT_TYPE                 0
CODE_GENDER                        0
FLAG_OWN_CAR                       0
FLAG_OWN_REALTY                    0
CNT_CHILDREN                       0
AMT_INCOME_TOTAL                   0
AMT_CREDIT                         0
AMT_ANNUITY                       12
AMT_GOODS_PRICE                  278
NAME_TYPE_SUITE                 1292
NAME_INCOME_TYPE                   0
NAME_EDUCATION_TYPE                0
NAME_FAMILY_STATUS                 0
NAME_HOUSING_TYPE                  0
REGION_POPULATION_RELATIVE         0
DAYS_BIRTH                         0
DAYS_EMPLOYED                      0
DAYS_REGISTRATION                  0
DAYS_ID_PUBLISH                    0
OWN_CAR_AGE                   202929
FLAG_MOBIL                         0
FLAG_EMP_PHONE                     0
FLAG_WORK_PHONE                    0
FLAG_CONT_MOBILE                   0
FLAG_PHONE                         0
F

In [ ]:
nan_precents = (application_train.isnull().sum()/application_train.isnull().count()*100)
feature_overview_df  = pd.concat([total_nans, nan_precents], axis=1, keys=['NaN Count', 'NaN Pencent'])
feature_overview_df['Type'] = [application_train[c].dtype for c in feature_overview_df.index]
pd.set_option('display.max_rows', None)
display(feature_overview_df)
pd.set_option('display.max_rows', 20)

In [ ]:
all_application_is_nan_df = pd.DataFrame()
for column in application_train.columns:
    if application_train[column].isnull().sum() == 0:
        continue
    all_application_is_nan_df['is_nan_' + column] = application_train[column].isnull()
    all_application_is_nan_df['is_nan_' + column] = all_application_is_nan_df['is_nan_' + column].map(lambda v: 1 if v else 0)
all_application_is_nan_df['target'] = application_train['TARGET']
all_application_is_nan_df = all_application_is_nan_df[pd.notnull(all_application_is_nan_df['target'])]

In [ ]:
display(all_application_is_nan_df)

In [ ]:
Y = all_application_is_nan_df.pop('target')
X = all_application_is_nan_df

In [ ]:
train_X, valid_X, train_Y, valid_Y = train_test_split(X, Y, test_size=0.2, random_state=2018)

In [ ]:
clf = LGBMClassifier(n_estimators=200, learning_rate=0.01)

In [ ]:
clf.fit(
        train_X,
        train_Y,
        eval_set=[(train_X, train_Y), (valid_X, valid_Y)],
        eval_metric='auc',
        early_stopping_rounds=50,
        verbose=False
       )

In [ ]:
plot_importance(clf, figsize=(10,10))

In [ ]:
#print("only showing the distribution for the first few columns, edit the counter to show all distribution")
#show_feature_count = 10
#for column in all_application_df.columns:
#   if show_feature_count == 0:
#        break
#    show_feature_count -= 1
#    draw_feature_distribution(all_application_df, column)

In [ ]:
draw_feature_distribution(application_train, 'TARGET')

In [ ]:
draw_feature_distribution(application_train, 'DAYS_EMPLOYED')
# ToDo(JamesBalcomb): fix "ValueError: max() arg is an empty sequence" - add check for 'class_t_values'

In [ ]:
#EXT_SOURCE_1
#EXT_SOURCE_2
#EXT_SOURCE_3

In [ ]:
draw_feature_distribution(application_train, 'EXT_SOURCE_1')

In [ ]:
draw_feature_distribution(application_train, 'EXT_SOURCE_2')

In [ ]:
draw_feature_distribution(application_train, 'EXT_SOURCE_3')

# EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3

Q: Is there a relationship between any of these three continuous variables and the binary classification target variable?
A: Yes, but EXT_SOURCE_2 is oddly shapen.

In [ ]:
# Seaborn Violin Plot - correlation; distribution and density
sns.violinplot(x='TARGET', y='EXT_SOURCE_1', data=application_train)

In [ ]:
# Seaborn Violin Plot - correlation; distribution and density
sns.violinplot(x='TARGET', y='EXT_SOURCE_2', data=application_train)

In [ ]:
# Seaborn Violin Plot - correlation; distribution and density
sns.violinplot(x='TARGET', y='EXT_SOURCE_3', data=application_train)

In [ ]:
application_train.hist(column='EXT_SOURCE_1', # Column to plot
              figsize=(8,8),                  # Plot size
              color="blue"                    # Plot color
              )

In [ ]:
application_train.hist(column='EXT_SOURCE_2', # Column to plot
              figsize=(8,8),                  # Plot size
              color="blue"                    # Plot color
              )

In [ ]:
application_train.hist(column='EXT_SOURCE_3', # Column to plot
              figsize=(8,8),                  # Plot size
              color="blue"                    # Plot color
              )

In [ ]:
# https://gist.github.com/ltfschoen/4c5d2cf26b8be5355043273493a6b8b9#file-proportions_of_missing_data_in_dataframe_columns-py
def get_percentage_missing(series):
    """ Calculates percentage of NaN values in DataFrame
    :param series: Pandas DataFrame object
    :return: float
    """
    num = series.isnull().sum()
    den = len(series)
    return round(num / den, 2)

In [ ]:
get_percentage_missing(application_train['EXT_SOURCE_1'])

In [ ]:
get_percentage_missing(application_train['EXT_SOURCE_2'])

In [ ]:
get_percentage_missing(application_train['EXT_SOURCE_3'])

In [ ]:
# # https://datascience.stackexchange.com/questions/12645/how-to-count-the-number-of-missing-values-in-each-row-in-pandas-dataframe
# # Count of Missing Values per Column
# df.isnull().sum(axis=0)
# # Count of Missing Values per Row
# df.isnull().sum(axis=1)

In [ ]:
# https://towardsdatascience.com/the-tale-of-missing-values-in-python-c96beb0e8a9d
# If the missing value isn’t identified as NaN , then we have to first convert or replace such non NaN entry with a NaN.
data_name[‘column_name’].replace(0, np.nan, inplace= True)

In [ ]:
# https://gist.github.com/ltfschoen/4c5d2cf26b8be5355043273493a6b8b9#file-proportions_of_missing_data_in_dataframe_columns-py
df = application_train
# Only include columns that contain any NaN values
df_with_any_null_values = df[df.columns[df.isnull().any()].tolist()]

get_percentage_missing(df_with_any_null_values)

# Iterate over columns in DataFrame and delete those with where >30% of the values are null/NaN
for name, values in df_with_any_null_values.iteritems():
    # print("%r: %r" % (name, values))
    if get_percentage_missing(df_with_any_null_values[name]) > 0.30:
        print("Deleting Column %r: " % (name))
        # df_with_any_null_values.drop(name, axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of EXT_SOURCE_1")
ax = sns.distplot(application_train["EXT_SOURCE_1"].dropna())

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of EXT_SOURCE_2")
ax = sns.distplot(application_train["EXT_SOURCE_2"].dropna())

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of EXT_SOURCE_3")
ax = sns.distplot(application_train["EXT_SOURCE_3"].dropna())

In [ ]:
application_train['EXT_SOURCE_AVG'] = application_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of EXT_SOURCE_AVG")
ax = sns.distplot(application_train["EXT_SOURCE_AVG"].dropna())

In [ ]:
# https://stackoverflow.com/questions/35277075/python-pandas-counting-the-occurrences-of-a-specific-value
#df.loc[df.education == '9th', 'education'].count()
#(df.education == '9th').sum()
#df.query('education == "9th"').education.count()

In [ ]:
application_train.loc[application_train.EXT_SOURCE_1 == 0.0, 'EXT_SOURCE_1'].count()

In [ ]:
application_train.loc[application_train.EXT_SOURCE_2 == 0.0, 'EXT_SOURCE_2'].count()

In [ ]:
application_train.loc[application_train.EXT_SOURCE_3 == 0.0, 'EXT_SOURCE_3'].count()

In [ ]:
application_train['EXT_SOURCE_1'].value_counts()

In [ ]:
application_train['EXT_SOURCE_2'].value_counts()

In [ ]:
application_train['EXT_SOURCE_3'].value_counts()
# application_train['EXT_SOURCE_3'].value_counts()[:20]
# # ValueError: index must be monotonic increasing or decreasing

In [ ]:
application_train['EXT_SOURCE_1'].describe()

In [ ]:
application_train['EXT_SOURCE_2'].describe()

In [ ]:
application_train['EXT_SOURCE_3'].describe()

In [ ]:
application_train['EXT_SOURCE_1'].plot(kind='hist', bins=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])

In [ ]:
application_train['EXT_SOURCE_1'].plot(kind='hist', bins=[0.0,0.2,0.4,0.6,0.8,1.0])

In [ ]:
application_train['EXT_SOURCE_2'].plot(kind='hist', bins=[0.0,0.2,0.4,0.6,0.8,1.0])

In [ ]:
application_train['EXT_SOURCE_3'].plot(kind='hist', bins=[0.0,0.2,0.4,0.6,0.8,1.0])

In [ ]:
# https://community.modeanalytics.com/python/tutorial/python-histograms-boxplots-and-distributions/
bin_values = np.arange(start=0, stop=1, step=0.2)
us_mq_airlines_index = application_train['TARGET'].isin(['US','MQ']) # create index
us_mq_airlines = application_train[us_mq_airlines_index] # select rows
group_carriers = us_mq_airlines.groupby('TARGET')['EXT_SOURCE_1'] # group values
group_carriers.plot(kind='hist', bins=bin_values, figsize=[12,6], alpha=.4, legend=True) # alpha for transparency

In [ ]:
application_train['EXT_SOURCE_1'].plot(kind='box', figsize=[16,8])

In [ ]:
application_train['EXT_SOURCE_2'].plot(kind='box', figsize=[16,8])

In [ ]:
application_train['EXT_SOURCE_3'].plot(kind='box', figsize=[16,8])

In [ ]:
# Define a function to create the scatterplot. This makes it easy to
# reuse code within and across notebooks
def scatterplot(x_data, y_data, x_label, y_label, title):

    # Create the plot object
    _, ax = plt.subplots()

    # Plot the data, set the size (s), color and transparency (alpha) of the points
    ax.scatter(x_data, y_data, s = 30, color = '#539caf', alpha = 0.75)

    # Label the axes and provide a title
    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)

In [ ]:
## Call the function to create plot
#scatterplot(x_data = daily_data['temp']
#            , y_data = daily_data['cnt']
#            , x_label = 'Normalized temperature (C)'
#            , y_label = 'Check outs'
#            , title = 'Number of Check Outs vs Temperature')

In [ ]:
# Call the function to create plot
scatterplot(x_data = application_train['EXT_SOURCE_1']
            , y_data = application_train['EXT_SOURCE_2']
            , x_label = 'EXT_SOURCE_1'
            , y_label = 'EXT_SOURCE_2'
            , title = 'EXT_SOURCE_1 vs. EXT_SOURCE_2'
           )

In [ ]:
# Call the function to create plot
scatterplot(x_data = application_train['EXT_SOURCE_1']
            , y_data = application_train['EXT_SOURCE_3']
            , x_label = 'EXT_SOURCE_1'
            , y_label = 'EXT_SOURCE_2'
            , title = 'EXT_SOURCE_1 vs. EXT_SOURCE_3'
           )

In [ ]:
# Call the function to create plot
scatterplot(x_data = application_train['EXT_SOURCE_2']
            , y_data = application_train['EXT_SOURCE_3']
            , x_label = 'EXT_SOURCE_2'
            , y_label = 'EXT_SOURCE_3'
            , title = 'EXT_SOURCE_2 vs. EXT_SOURCE_3'
           )

In [ ]:
application_train__ext_source_1__dropna = application_train.loc[:,['EXT_SOURCE_1','TARGET']]
application_train__ext_source_1__dropna.dropna(inplace = True)
scipy.stats.pointbiserialr(application_train__ext_source_1__dropna['EXT_SOURCE_1'], application_train__ext_source_1__dropna['TARGET'])

In [ ]:
application_train__ext_source_2__dropna = application_train.loc[:,['EXT_SOURCE_2','TARGET']]
application_train__ext_source_2__dropna.dropna(inplace = True)
scipy.stats.pointbiserialr(application_train__ext_source_2__dropna['EXT_SOURCE_2'], application_train__ext_source_2__dropna['TARGET'])

In [ ]:
application_train__ext_source_3__dropna = application_train.loc[:,['EXT_SOURCE_3','TARGET']]
application_train__ext_source_3__dropna.dropna(inplace = True)
scipy.stats.pointbiserialr(application_train__ext_source_3__dropna['EXT_SOURCE_3'], application_train__ext_source_3__dropna['TARGET'])

In [ ]:
scipy.stats.pearsonr(application_train__ext_source_1__dropna['EXT_SOURCE_1'], application_train__ext_source_1__dropna['TARGET'])

In [ ]:
scipy.stats.pearsonr(application_train__ext_source_2__dropna['EXT_SOURCE_2'], application_train__ext_source_2__dropna['TARGET'])

In [ ]:
scipy.stats.pearsonr(application_train__ext_source_3__dropna['EXT_SOURCE_3'], application_train__ext_source_3__dropna['TARGET'])

In [ ]:
np.corrcoef(application_train__ext_source_1__dropna['EXT_SOURCE_1'], application_train__ext_source_1__dropna['TARGET'])

In [ ]:
np.corrcoef(application_train__ext_source_2__dropna['EXT_SOURCE_2'], application_train__ext_source_2__dropna['TARGET'])

In [ ]:
np.corrcoef(application_train__ext_source_3__dropna['EXT_SOURCE_3'], application_train__ext_source_3__dropna['TARGET'])

In [ ]:
temp = previous_application["NAME_CONTRACT_TYPE"].value_counts()
fig = {
  "data": [
    {
      "values": temp.values,
      "labels": temp.index,
      "domain": {"x": [0, .48]},
      #"name": "Types of Loans",
      #"hoverinfo":"label+percent+name",
      "hole": .7,
      "type": "pie"
    },
    
    ],
  "layout": {
        "title":"Contract product type of previous application",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Contract product type",
                "x": 0.12,
                "y": 0.5
            }
            
        ]
    }
}
iplot(fig, filename='donut')

# Pearson Correlation of features

In [ ]:
data = [
    go.Heatmap(
        z=application_train.corr().values,
        x=application_train.columns.values,
        y=application_train.columns.values,
        colorscale='Viridis',
        reversescale = False,
        text = True ,
        opacity = 1.0 )
]

layout = go.Layout(
    title='Pearson Correlation of features',
    xaxis = dict(ticks='', nticks=36),
    yaxis = dict(ticks='' ),
    width = 900, height = 700,
margin=dict(
    l=240,
),)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')

# Multicollinearity
https://stackoverflow.com/questions/25676145/capturing-high-multi-collinearity-in-statsmodels/44012251#44012251
https://stackoverflow.com/questions/25676145/capturing-high-multi-collinearity-in-statsmodels/25833792#25833792
https://onlinecourses.science.psu.edu/stat501/node/347/


In [ ]:
## https://stackoverflow.com/questions/25676145/capturing-high-multi-collinearity-in-statsmodels/44012251#44012251
##...looking for a single number that captured the collinearity
##...options include the determinant and condition number of the correlation matrix
##...determinant of the correlation matrix will "range from 0 (Perfect Collinearity) to 1 (No Collinearity)"

In [ ]:
# Compute correlation matrices
pearson_product_moment_correlation_coefficients = np.corrcoef(application_train, rowvar=0)
## https://docs.scipy.org/doc/numpy/reference/generated/numpy.corrcoef.html
## Return Pearson product-moment correlation coefficients.

In [ ]:
# Compare the determinants
print np.linalg.det(pearson_product_moment_correlation_coefficients)

In [ ]:
# the condition number of the covariance matrix will approach infinity with perfect linear dependence
print np.linalg.cond(pearson_product_moment_correlation_coefficients)

In [ ]:
application_train.info()

In [ ]:
application_train.info(memory_usage='deep')

In [ ]:
application_train.memory_usage()

In [ ]:
application_train.memory_usage(index=False)

In [ ]:
# https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219
# Group loans by client id and calculate mean, max, min of loans
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']

# Merge with the clients dataframe
stats = clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left')

stats.head(10)

In [98]:
#F      202448
#M      105059
#XNA         4
#Name: CODE_GENDER, dtype: int64
202448 / (202448 + 105059), 105059 / (202448 + 105059)

(0.658352492788782, 0.34164750721121795)

In [99]:
#F      202448
#M      105059
#XNA         4
#Name: CODE_GENDER, dtype: int64
(4 * 0.658352492788782), (4 * 0.34164750721121795)

(2.633409971155128, 1.3665900288448718)

In [100]:
# Female: 202448 (0.6584) Male: 105059 (0.3417)